# Stock Index Prediction using Deep Learning Models

This notebook implements LSTM, GRU, and RNN models to predict the future prices of SP500 and IBEX35 indices.

## 1. Importing Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from datetime import datetime, timedelta

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Data Collection

We'll load the data from the CSV files created in the previous notebook.

In [6]:
# Check if CSV files exist, otherwise download data
if os.path.exists('sp500_data.csv') and os.path.exists('ibex35_data.csv'):
    sp500_data = pd.read_csv('sp500_data.csv', index_col='Date', parse_dates=True)
    ibex35_data = pd.read_csv('ibex35_data.csv', index_col='Date', parse_dates=True)
    print("Data loaded from CSV files.")
else:
    # Define the symbols of the indices

    sp500_ticker = "^GSPC"  # S&P 500
    ibex35_ticker = "^IBEX"  # IBEX 35
    
    # Define the date range
    start_date = "2000-01-01"
    end_date = "2024-01-01"
    
    # Download historical data from Yahoo Finance
    #sp500_data = yf.download(sp500_ticker, start=start_date, end=end_date)
    sp500_data = yf.Tickers(sp500_ticker, ibex35_ticker).history(period="1d", start=start_date, end=end_date)["Close"]

    #ibex35_data = yf.download(ibex35_ticker, start=start_date, end=end_date)
    ibex35_data = yf.Ticker(ibex35_ticker).history(period="1d", start=start_date, end=end_date)["Close"]

    
    # Save the data to CSV
    sp500_data.to_csv("sp500_data.csv")
    ibex35_data.to_csv("ibex35_data.csv")
    print("Data downloaded and saved to CSV files.")

Data loaded from CSV files.


In [7]:
# Display the first few rows of each dataset
print("S&P 500 Data:")
print(sp500_data.head())
print("\nIBEX 35 Data:")
print(ibex35_data.head())

S&P 500 Data:
Empty DataFrame
Columns: [Close]
Index: []

IBEX 35 Data:
Empty DataFrame
Columns: [Close]
Index: []


## 3. Data Preparation for Deep Learning Models

For deep learning models, we need to prepare the data in a specific format. We'll create sequences of data for time series forecasting.

In [ ]:
# Function to create sequences for time series forecasting
def create_sequences(data, seq_length):
    """
    Create sequences of data for time series forecasting.
    
    Parameters:
    data (numpy.ndarray): Input data array
    seq_length (int): Length of each sequence
    
    Returns:
    tuple: (X, y) where X is the input sequences and y is the target values
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0])  # Predict the Close price
    return np.array(X), np.array(y)

In [ ]:
# Prepare data for SP500
def prepare_data(data, seq_length=60, test_size=0.2, feature_columns=None):
    """
    Prepare data for deep learning models.
    
    Parameters:
    data (pandas.DataFrame): Input data
    seq_length (int): Length of each sequence
    test_size (float): Proportion of data to use for testing
    feature_columns (list): List of column names to use as features
    
    Returns:
    tuple: (X_train, X_test, y_train, y_test, scaler)
    """
    # Select features
    if feature_columns is None:
        # Use only OHLC and Volume as features by default
        feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    
    data_features = data[feature_columns].values
    
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data_features)
    
    # Create sequences
    X, y = create_sequences(data_scaled, seq_length)
    
    # Split into training and testing sets
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Convert to PyTorch tensors
    X_train = torch.FloatTensor(X_train)
    X_test = torch.FloatTensor(X_test)
    y_train = torch.FloatTensor(y_train).reshape(-1, 1)
    y_test = torch.FloatTensor(y_test).reshape(-1, 1)
    
    return X_train, X_test, y_train, y_test, scaler

In [ ]:
# Prepare data for SP500 and IBEX35
seq_length = 60  # Use 60 days of data to predict the next day

# For SP500
X_train_sp500, X_test_sp500, y_train_sp500, y_test_sp500, scaler_sp500 = prepare_data(
    sp500_data, seq_length=seq_length
)

# For IBEX35
X_train_ibex35, X_test_ibex35, y_train_ibex35, y_test_ibex35, scaler_ibex35 = prepare_data(
    ibex35_data, seq_length=seq_length
)

# Print the shapes of the data
print(f"SP500 - X_train: {X_train_sp500.shape}, X_test: {X_test_sp500.shape}, y_train: {y_train_sp500.shape}, y_test: {y_test_sp500.shape}")
print(f"IBEX35 - X_train: {X_train_ibex35.shape}, X_test: {X_test_ibex35.shape}, y_train: {y_train_ibex35.shape}, y_test: {y_test_ibex35.shape}")

In [ ]:
# Create DataLoader for batch processing
def create_dataloader(X, y, batch_size=32, shuffle=True):
    """
    Create DataLoader for batch processing.
    
    Parameters:
    X (torch.Tensor): Input data
    y (torch.Tensor): Target data
    batch_size (int): Batch size
    shuffle (bool): Whether to shuffle the data
    
    Returns:
    torch.utils.data.DataLoader: DataLoader object
    """
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

# Create DataLoaders
batch_size = 32

train_loader_sp500 = create_dataloader(X_train_sp500, y_train_sp500, batch_size=batch_size)
test_loader_sp500 = create_dataloader(X_test_sp500, y_test_sp500, batch_size=batch_size, shuffle=False)

train_loader_ibex35 = create_dataloader(X_train_ibex35, y_train_ibex35, batch_size=batch_size)
test_loader_ibex35 = create_dataloader(X_test_ibex35, y_test_ibex35, batch_size=batch_size, shuffle=False)

## 4. LSTM Model Implementation

Long Short-Term Memory (LSTM) networks are a type of recurrent neural network capable of learning order dependence in sequence prediction problems.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        """
        LSTM model for time series forecasting.
        
        Parameters:
        input_size (int): Number of features in the input
        hidden_size (int): Number of features in the hidden state
        num_layers (int): Number of recurrent layers
        output_size (int): Number of features in the output
        dropout (float): Dropout probability
        """
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Get the output from the last time step
        out = self.fc(out[:, -1, :])
        
        return out

In [ ]:
# Function to train the model
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=100, early_stopping_patience=10):
    """
    Train the model.
    
    Parameters:
    model (torch.nn.Module): Model to train
    train_loader (torch.utils.data.DataLoader): Training data loader
    test_loader (torch.utils.data.DataLoader): Testing data loader
    criterion (torch.nn.Module): Loss function
    optimizer (torch.optim.Optimizer): Optimizer
    num_epochs (int): Number of epochs to train for
    early_stopping_patience (int): Number of epochs to wait for improvement before stopping
    
    Returns:
    tuple: (model, train_losses, test_losses)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    train_losses = []
    test_losses = []
    best_test_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Forward pass
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            
            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * X_batch.size(0)
        
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        
        # Testing
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                # Forward pass
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                
                test_loss += loss.item() * X_batch.size(0)
        
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}')
        
        # Early stopping
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f'Early stopping at epoch {epoch+1}')
                break
    
    # Load the best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, train_losses, test_losses

In [ ]:
# Function to evaluate the model
def evaluate_model(model, test_loader, scaler, feature_columns):
    """
    Evaluate the model.
    
    Parameters:
    model (torch.nn.Module): Model to evaluate
    test_loader (torch.utils.data.DataLoader): Testing data loader
    scaler (sklearn.preprocessing.MinMaxScaler): Scaler used to normalize the data
    feature_columns (list): List of column names used as features
    
    Returns:
    tuple: (y_true, y_pred, metrics)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            
            # Forward pass
            batch_pred = model(X_batch)
            
            # Move to CPU for numpy conversion
            y_batch = y_batch.cpu().numpy()
            batch_pred = batch_pred.cpu().numpy()
            
            # Inverse transform to get actual prices
            # Create dummy arrays with the same shape as the original data
            dummy = np.zeros((len(y_batch), len(feature_columns)))
            dummy[:, 3] = y_batch.flatten()  # Assuming Close is at index 3
            y_true_inv = scaler.inverse_transform(dummy)[:, 3]
            
            dummy[:, 3] = batch_pred.flatten()
            y_pred_inv = scaler.inverse_transform(dummy)[:, 3]
            
            y_true.extend(y_true_inv)
            y_pred.extend(y_pred_inv)
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    
    return y_true, y_pred, metrics

In [ ]:
# Function to plot the results
def plot_results(y_true, y_pred, title):
    """
    Plot the true vs predicted values.
    
    Parameters:
    y_true (numpy.ndarray): True values
    y_pred (numpy.ndarray): Predicted values
    title (str): Plot title
    """
    plt.figure(figsize=(12, 6))
    plt.plot(y_true, label='True')
    plt.plot(y_pred, label='Predicted')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

In [ ]:
# Train LSTM model for SP500
input_size = X_train_sp500.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2

lstm_model_sp500 = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

criterion = nn.MSELoss()
optimizer = optim.Adam(lstm_model_sp500.parameters(), lr=0.001)

lstm_model_sp500, train_losses_sp500, test_losses_sp500 = train_model(
    lstm_model_sp500, train_loader_sp500, test_loader_sp500, criterion, optimizer, num_epochs=100
)

In [ ]:
# Evaluate LSTM model for SP500
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
y_true_sp500, y_pred_sp500, metrics_sp500 = evaluate_model(
    lstm_model_sp500, test_loader_sp500, scaler_sp500, feature_columns
)

print("LSTM Model Metrics for SP500:")
for metric, value in metrics_sp500.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_sp500, y_pred_sp500, 'LSTM Model Predictions for SP500')

In [ ]:
# Train LSTM model for IBEX35
input_size = X_train_ibex35.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2

lstm_model_ibex35 = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

criterion = nn.MSELoss()
optimizer = optim.Adam(lstm_model_ibex35.parameters(), lr=0.001)

lstm_model_ibex35, train_losses_ibex35, test_losses_ibex35 = train_model(
    lstm_model_ibex35, train_loader_ibex35, test_loader_ibex35, criterion, optimizer, num_epochs=100
)

In [ ]:
# Evaluate LSTM model for IBEX35
y_true_ibex35, y_pred_ibex35, metrics_ibex35 = evaluate_model(
    lstm_model_ibex35, test_loader_ibex35, scaler_ibex35, feature_columns
)

print("LSTM Model Metrics for IBEX35:")
for metric, value in metrics_ibex35.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_ibex35, y_pred_ibex35, 'LSTM Model Predictions for IBEX35')

## 5. GRU Model Implementation

Gated Recurrent Unit (GRU) is a type of recurrent neural network that is similar to LSTM but has fewer parameters.

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        """
        GRU model for time series forecasting.
        
        Parameters:
        input_size (int): Number of features in the input
        hidden_size (int): Number of features in the hidden state
        num_layers (int): Number of recurrent layers
        output_size (int): Number of features in the output
        dropout (float): Dropout probability
        """
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # GRU layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        
        # Get the output from the last time step
        out = self.fc(out[:, -1, :])
        
        return out

In [ ]:
# Train GRU model for SP500
input_size = X_train_sp500.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2

gru_model_sp500 = GRUModel(input_size, hidden_size, num_layers, output_size, dropout)

criterion = nn.MSELoss()
optimizer = optim.Adam(gru_model_sp500.parameters(), lr=0.001)

gru_model_sp500, train_losses_sp500_gru, test_losses_sp500_gru = train_model(
    gru_model_sp500, train_loader_sp500, test_loader_sp500, criterion, optimizer, num_epochs=100
)

In [ ]:
# Evaluate GRU model for SP500
y_true_sp500_gru, y_pred_sp500_gru, metrics_sp500_gru = evaluate_model(
    gru_model_sp500, test_loader_sp500, scaler_sp500, feature_columns
)

print("GRU Model Metrics for SP500:")
for metric, value in metrics_sp500_gru.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_sp500_gru, y_pred_sp500_gru, 'GRU Model Predictions for SP500')

In [ ]:
# Train GRU model for IBEX35
input_size = X_train_ibex35.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2

gru_model_ibex35 = GRUModel(input_size, hidden_size, num_layers, output_size, dropout)

criterion = nn.MSELoss()
optimizer = optim.Adam(gru_model_ibex35.parameters(), lr=0.001)

gru_model_ibex35, train_losses_ibex35_gru, test_losses_ibex35_gru = train_model(
    gru_model_ibex35, train_loader_ibex35, test_loader_ibex35, criterion, optimizer, num_epochs=100
)

In [ ]:
# Evaluate GRU model for IBEX35
y_true_ibex35_gru, y_pred_ibex35_gru, metrics_ibex35_gru = evaluate_model(
    gru_model_ibex35, test_loader_ibex35, scaler_ibex35, feature_columns
)

print("GRU Model Metrics for IBEX35:")
for metric, value in metrics_ibex35_gru.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_ibex35_gru, y_pred_ibex35_gru, 'GRU Model Predictions for IBEX35')

## 6. RNN Model Implementation

Simple Recurrent Neural Network (RNN) implementation.

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        """
        RNN model for time series forecasting.
        
        Parameters:
        input_size (int): Number of features in the input
        hidden_size (int): Number of features in the hidden state
        num_layers (int): Number of recurrent layers
        output_size (int): Number of features in the output
        dropout (float): Dropout probability
        """
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate RNN
        out, _ = self.rnn(x, h0)
        
        # Get the output from the last time step
        out = self.fc(out[:, -1, :])
        
        return out

In [ ]:
# Train RNN model for SP500
input_size = X_train_sp500.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2

rnn_model_sp500 = RNNModel(input_size, hidden_size, num_layers, output_size, dropout)

criterion = nn.MSELoss()
optimizer = optim.Adam(rnn_model_sp500.parameters(), lr=0.001)

rnn_model_sp500, train_losses_sp500_rnn, test_losses_sp500_rnn = train_model(
    rnn_model_sp500, train_loader_sp500, test_loader_sp500, criterion, optimizer, num_epochs=100
)

In [ ]:
# Evaluate RNN model for SP500
y_true_sp500_rnn, y_pred_sp500_rnn, metrics_sp500_rnn = evaluate_model(
    rnn_model_sp500, test_loader_sp500, scaler_sp500, feature_columns
)

print("RNN Model Metrics for SP500:")
for metric, value in metrics_sp500_rnn.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_sp500_rnn, y_pred_sp500_rnn, 'RNN Model Predictions for SP500')

In [ ]:
# Train RNN model for IBEX35
input_size = X_train_ibex35.shape[2]  # Number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2

rnn_model_ibex35 = RNNModel(input_size, hidden_size, num_layers, output_size, dropout)

criterion = nn.MSELoss()
optimizer = optim.Adam(rnn_model_ibex35.parameters(), lr=0.001)

rnn_model_ibex35, train_losses_ibex35_rnn, test_losses_ibex35_rnn = train_model(
    rnn_model_ibex35, train_loader_ibex35, test_loader_ibex35, criterion, optimizer, num_epochs=100
)

In [ ]:
# Evaluate RNN model for IBEX35
y_true_ibex35_rnn, y_pred_ibex35_rnn, metrics_ibex35_rnn = evaluate_model(
    rnn_model_ibex35, test_loader_ibex35, scaler_ibex35, feature_columns
)

print("RNN Model Metrics for IBEX35:")
for metric, value in metrics_ibex35_rnn.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_ibex35_rnn, y_pred_ibex35_rnn, 'RNN Model Predictions for IBEX35')

## 7. Ensemble Model Implementation

Create an ensemble model by combining the predictions from LSTM, GRU, and RNN models.

In [ ]:
# Function to create ensemble predictions
def ensemble_predictions(models, test_loader, scaler, feature_columns, weights=None):
    """
    Create ensemble predictions by combining the predictions from multiple models.
    
    Parameters:
    models (list): List of models
    test_loader (torch.utils.data.DataLoader): Testing data loader
    scaler (sklearn.preprocessing.MinMaxScaler): Scaler used to normalize the data
    feature_columns (list): List of column names used as features
    weights (list): List of weights for each model (default: equal weights)
    
    Returns:
    tuple: (y_true, y_pred, metrics)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Set models to evaluation mode
    for model in models:
        model.to(device)
        model.eval()
    
    # Set equal weights if not provided
    if weights is None:
        weights = [1/len(models)] * len(models)
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            
            # Get predictions from each model
            batch_preds = []
            for model in models:
                batch_pred = model(X_batch).cpu().numpy()
                batch_preds.append(batch_pred)
            
            # Combine predictions using weighted average
            ensemble_pred = np.zeros_like(batch_preds[0])
            for i, pred in enumerate(batch_preds):
                ensemble_pred += weights[i] * pred
            
            # Move to CPU for numpy conversion
            y_batch = y_batch.cpu().numpy()
            
            # Inverse transform to get actual prices
            # Create dummy arrays with the same shape as the original data
            dummy = np.zeros((len(y_batch), len(feature_columns)))
            dummy[:, 3] = y_batch.flatten()  # Assuming Close is at index 3
            y_true_inv = scaler.inverse_transform(dummy)[:, 3]
            
            dummy[:, 3] = ensemble_pred.flatten()
            y_pred_inv = scaler.inverse_transform(dummy)[:, 3]
            
            y_true.extend(y_true_inv)
            y_pred.extend(y_pred_inv)
    
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate metrics
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }
    
    return y_true, y_pred, metrics

In [ ]:
# Create ensemble predictions for SP500
models_sp500 = [lstm_model_sp500, gru_model_sp500, rnn_model_sp500]

# Use weights based on individual model performance (inverse of MSE)
mse_values = [metrics_sp500['MSE'], metrics_sp500_gru['MSE'], metrics_sp500_rnn['MSE']]
weights = [1/mse for mse in mse_values]
weights = [w/sum(weights) for w in weights]  # Normalize weights

print(f"Ensemble weights for SP500: {weights}")

y_true_sp500_ensemble, y_pred_sp500_ensemble, metrics_sp500_ensemble = ensemble_predictions(
    models_sp500, test_loader_sp500, scaler_sp500, feature_columns, weights=weights
)

print("\nEnsemble Model Metrics for SP500:")
for metric, value in metrics_sp500_ensemble.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_sp500_ensemble, y_pred_sp500_ensemble, 'Ensemble Model Predictions for SP500')

In [ ]:
# Create ensemble predictions for IBEX35
models_ibex35 = [lstm_model_ibex35, gru_model_ibex35, rnn_model_ibex35]

# Use weights based on individual model performance (inverse of MSE)
mse_values = [metrics_ibex35['MSE'], metrics_ibex35_gru['MSE'], metrics_ibex35_rnn['MSE']]
weights = [1/mse for mse in mse_values]
weights = [w/sum(weights) for w in weights]  # Normalize weights

print(f"Ensemble weights for IBEX35: {weights}")

y_true_ibex35_ensemble, y_pred_ibex35_ensemble, metrics_ibex35_ensemble = ensemble_predictions(
    models_ibex35, test_loader_ibex35, scaler_ibex35, feature_columns, weights=weights
)

print("\nEnsemble Model Metrics for IBEX35:")
for metric, value in metrics_ibex35_ensemble.items():
    print(f"{metric}: {value:.6f}")

# Plot the results
plot_results(y_true_ibex35_ensemble, y_pred_ibex35_ensemble, 'Ensemble Model Predictions for IBEX35')

## 8. Compare Model Performance

In [ ]:
# Compare model performance for SP500
models_sp500 = ['LSTM', 'GRU', 'RNN', 'Ensemble']
metrics_list_sp500 = [metrics_sp500, metrics_sp500_gru, metrics_sp500_rnn, metrics_sp500_ensemble]

# Create a DataFrame for comparison
comparison_sp500 = pd.DataFrame(index=models_sp500)
for metric in ['MSE', 'RMSE', 'MAE', 'R2']:
    comparison_sp500[metric] = [metrics[metric] for metrics in metrics_list_sp500]

print("Model Performance Comparison for SP500:")
print(comparison_sp500)

# Plot the comparison
plt.figure(figsize=(12, 8))
for i, metric in enumerate(['MSE', 'RMSE', 'MAE']):
    plt.subplot(3, 1, i+1)
    plt.bar(models_sp500, comparison_sp500[metric])
    plt.title(f'{metric} Comparison for SP500')
    plt.ylabel(metric)
plt.tight_layout()
plt.show()

# Plot R2 separately (higher is better)
plt.figure(figsize=(8, 4))
plt.bar(models_sp500, comparison_sp500['R2'])
plt.title('R2 Comparison for SP500')
plt.ylabel('R2')
plt.show()

In [ ]:
# Compare model performance for IBEX35
models_ibex35 = ['LSTM', 'GRU', 'RNN', 'Ensemble']
metrics_list_ibex35 = [metrics_ibex35, metrics_ibex35_gru, metrics_ibex35_rnn, metrics_ibex35_ensemble]

# Create a DataFrame for comparison
comparison_ibex35 = pd.DataFrame(index=models_ibex35)
for metric in ['MSE', 'RMSE', 'MAE', 'R2']:
    comparison_ibex35[metric] = [metrics[metric] for metrics in metrics_list_ibex35]

print("Model Performance Comparison for IBEX35:")
print(comparison_ibex35)

# Plot the comparison
plt.figure(figsize=(12, 8))
for i, metric in enumerate(['MSE', 'RMSE', 'MAE']):
    plt.subplot(3, 1, i+1)
    plt.bar(models_ibex35, comparison_ibex35[metric])
    plt.title(f'{metric} Comparison for IBEX35')
    plt.ylabel(metric)
plt.tight_layout()
plt.show()

# Plot R2 separately (higher is better)
plt.figure(figsize=(8, 4))
plt.bar(models_ibex35, comparison_ibex35['R2'])
plt.title('R2 Comparison for IBEX35')
plt.ylabel('R2')
plt.show()

## 9. Save Models for Streamlit App

In [ ]:
# Create a directory for models if it doesn't exist
if not os.path.exists('models'):
    os.makedirs('models')

# Function to save a model
def save_model(model, filename):
    """
    Save a PyTorch model.
    
    Parameters:
    model (torch.nn.Module): Model to save
    filename (str): Filename to save the model to
    """
    torch.save(model.state_dict(), filename)
    print(f"Model saved to {filename}")

# Save SP500 models
save_model(lstm_model_sp500, 'models/lstm_sp500.pth')
save_model(gru_model_sp500, 'models/gru_sp500.pth')
save_model(rnn_model_sp500, 'models/rnn_sp500.pth')

# Save IBEX35 models
save_model(lstm_model_ibex35, 'models/lstm_ibex35.pth')
save_model(gru_model_ibex35, 'models/gru_ibex35.pth')
save_model(rnn_model_ibex35, 'models/rnn_ibex35.pth')

# Save model parameters for later use
model_params = {
    'input_size': input_size,
    'hidden_size': hidden_size,
    'num_layers': num_layers,
    'output_size': output_size,
    'dropout': dropout,
    'seq_length': seq_length,
    'feature_columns': feature_columns,
    'ensemble_weights_sp500': weights,
    'ensemble_weights_ibex35': weights
}

with open('models/model_params.pkl', 'wb') as f:
    pickle.dump(model_params, f)
print("Model parameters saved to models/model_params.pkl")

# Save scalers
with open('models/scaler_sp500.pkl', 'wb') as f:
    pickle.dump(scaler_sp500, f)
print("SP500 scaler saved to models/scaler_sp500.pkl")

with open('models/scaler_ibex35.pkl', 'wb') as f:
    pickle.dump(scaler_ibex35, f)
print("IBEX35 scaler saved to models/scaler_ibex35.pkl")

## 10. Conclusion

In this notebook, we have implemented LSTM, GRU, and RNN models to predict the future prices of SP500 and IBEX35 indices. We have also created an ensemble model by combining the predictions from these models. The models have been saved for later use in the Streamlit app.